<a href="https://colab.research.google.com/github/aibak365/Machine-learning/blob/main/implementing_RNN_to_learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!curl -O https://download.pytorch.org/tutorial/data.zip; unzip data.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2814k  100 2814k    0     0  9022k      0 --:--:-- --:--:-- --:--:-- 9050k
Archive:  data.zip
   creating: data/
  inflating: data/eng-fra.txt        
   creating: data/names/
  inflating: data/names/Arabic.txt   
  inflating: data/names/Chinese.txt  
  inflating: data/names/Czech.txt    
  inflating: data/names/Dutch.txt    
  inflating: data/names/English.txt  
  inflating: data/names/French.txt   
  inflating: data/names/German.txt   
  inflating: data/names/Greek.txt    
  inflating: data/names/Irish.txt    
  inflating: data/names/Italian.txt  
  inflating: data/names/Japanese.txt  
  inflating: data/names/Korean.txt   
  inflating: data/names/Polish.txt   
  inflating: data/names/Portuguese.txt  
  inflating: data/names/Russian.txt  
  inflating: data/names/Scottish.txt  
  inflating: data/names/Spanish.txt  
  inflating

In [2]:
import os
import random
from string import ascii_letters

import torch
from torch import nn
import torch.nn.functional as F
from unidecode import unidecode
_ = torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
data_dir = "./data/names"

lang2label = {
    filepath.split('.')[0]: torch.tensor([i],dtype=torch.long)
    for i, filepath in enumerate(os.listdir(data_dir))
}
lang2label

{'Irish': tensor([0]),
 'Japanese': tensor([1]),
 'Chinese': tensor([2]),
 'Korean': tensor([3]),
 'Vietnamese': tensor([4]),
 'Arabic': tensor([5]),
 'German': tensor([6]),
 'English': tensor([7]),
 'French': tensor([8]),
 'Greek': tensor([9]),
 'Portuguese': tensor([10]),
 'Czech': tensor([11]),
 'Scottish': tensor([12]),
 'Russian': tensor([13]),
 'Dutch': tensor([14]),
 'Italian': tensor([15]),
 'Spanish': tensor([16]),
 'Polish': tensor([17])}

In [8]:
num_languages = len(lang2label)

In [10]:
char2idx = {letter: i for i, letter in enumerate(ascii_letters + " .,:;-'")}
num_letters = len(char2idx); num_letters

59

In [83]:
def name2tensor(name):
  tensor = torch.zeros(len(name),1,len(char2idx))
  for i, letter in enumerate(name):
    tensor[i][0][char2idx[letter]] = 1
  return tensor
name2tensor("aibak")

tensor([[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0.]],

        [[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       

In [12]:
tensor_names = []
target_langs = []
for file in os.listdir(data_dir):
  with open(os.path.join(data_dir,file)) as f:
    lang = file.split(".")[0]
    names = [unidecode(line.strip()) for line in f]
    for name in names:
      try:
        tensor_names.append(name2tensor(name))
        target_langs.append(lang2label[lang])
      except KeyError:
        pass


Streaming output truncated to the last 5000 lines.
f
r
e
d
o
n
i
a
M
a
n
t
o
v
a
n
i
M
a
r
c
h
e
g
i
a
n
o
M
a
r
c
h
e
s
i
M
a
r
c
h
e
t
t
i
M
a
r
c
h
i
o
n
i
M
a
r
c
o
n
i
M
a
r
i
M
a
r
i
a
M
a
r
i
a
n
i
M
a
r
i
n
o
M
a
r
m
o
M
a
r
t
e
l
l
i
M
a
r
t
i
n
e
l
l
i
M
a
s
i
M
a
s
i
n
M
a
z
z
a
M
e
r
l
o
M
e
s
s
a
n
a
M
i
c
h
e
l
i
M
i
l
a
n
i
M
i
l
a
n
o
M
o
d
u
g
n
o
M
o
n
d
a
d
o
r
i
M
o
n
d
o
M
o
n
t
a
g
n
a
M
o
n
t
a
n
a
M
o
n
t
a
n
a
r
i
M
o
n
t
e
M
o
n
t
i
M
o
r
a
n
d
i
M
o
r
e
l
l
o
M
o
r
e
t
t
i
M
o
r
r
a
M
o
s
c
h
e
l
l
a
M
o
s
c
o
n
i
M
o
t
t
a
M
u
g
g
i
a
M
u
r
a
r
o
M
u
r
g
i
a
M
u
r
t
a
s
N
a
c
a
r
N
a
g
g
i
N
a
g
g
i
a
N
a
l
d
i
N
a
n
a
N
a
n
i
N
a
n
n
i
N
a
n
n
i
n
i
N
a
p
o
l
e
o
n
i
N
a
p
o
l
e
t
a
n
i
N
a
p
o
l
i
e
l
l
o
N
a
r
d
i
N
a
r
d
o
N
a
r
d
o
v
i
n
o
N
a
s
a
t
o
N
a
s
c
i
m
b
e
n
e
N
a
s
c
i
m
b
e
n
i
N
a
t
a
l
e
N
a
v
e
N
a
z
a
r
i
o
N
e
c
c
h
i
N
e
g
r
i
N
e
g
r
i
n
i
N
e
l
l
i
N
e
n
c
i
N
e
p
i
N
e
r
i
N
e
r
o
n
i
N
e
r
v
e
t
t
i
N
e
r
v
i
N
e
s
p
o
l
a
N
i
c
a

In [13]:
print(tensor_names[0])
print(target_langs[0])

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0.]],

        [[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tensor_names, target_langs, test_size=0.2,shuffle=True ,random_state=42)
print(X_train[0],y_train[0])

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0.]],

        [[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       

In [15]:
print(len(X_train),len(X_test),len(y_train),len(y_test))

16056 4014 16056 4014


In [ ]:
num_langs = len(lang2label)
hidden_size = 256
learning_rate = 0.01
class simple_RNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(simple_RNN, self).__init__()
    self.hidden_size = hidden_size
    self.i2h = nn.Linear(input_size+hidden_size, hidden_size)
    self.i2o = nn.Linear(input_size+hidden_size, output_size)
  def forward(self, x,hidden_state):
    combined = torch.cat((x,hidden_state),1)
    hidden = self.i2h(combined)
    output = self.i2o(combined)
    return output, hidden
  def init_hidden(self):
    return nn.init.kaiming_uniform(torch.empty(1,self.hidden_size))

model = simple_RNN(num_letters, hidden_size, num_langs)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
epochs = 3
print_interval = 1000
for epoch in range(epochs):
  for i, (name, label) in enumerate(zip(X_train,y_train)):
    hidden = model.init_hidden()
    for letter in name:
      output, hidden = model(letter,hidden)
    loss = criterion(output, label)
    optimizer.zero_grad()
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), 1)
    optimizer.step()
    if (i+1) % print_interval == 0:
      print(f"Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(X_train)}], Loss: {loss.item():.4f}")

In [ ]:
accuracy = 0
model.eval()
with torch.no_grad():
  for name, label in zip(X_test,y_test):
    hidden = model.init_hidden()
    for letter in name:
      output, hidden = model(letter,hidden)
    _, predicted = torch.max(output.data, 1)
    if predicted == label:
      accuracy += 1
print(accuracy/len(X_test))

In [ ]:
label2lang = {label.item(): lang for lang, label in lang2label.items()}
print(label2lang)

In [ ]:
def predict_language(name):
  model.eval()
  tensor_name = name2tensor(name)
  with torch.no_grad():
    hidden = model.init_hidden()
    for letter in tensor_name:
      output, hidden = model(letter,hidden)
    _, predicted = torch.max(output, 1)
    print(f"Predicted: {predicted.item()}, Actual: {label.item()}")
    return label2lang[predicted.item()]

predict_language("Qin")

In [ ]:
class GRU(nn.Module):
    def __init__(self, num_layers, hidden_size):
        super(GRU, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.gru = nn.GRU(
            input_size=num_letters,
            hidden_size=hidden_size,
            num_layers=num_layers,
        )
        self.fc = nn.Linear(hidden_size, num_langs)

    def forward(self, x):
        hidden_state = self.init_hidden()
        output, hidden_state = self.gru(x, hidden_state)
        output = self.fc(output[-1])
        return output

    def init_hidden(self):
        return torch.zeros(self.num_layers, 1, self.hidden_size).to(device)

In [ ]:
model = GRU(num_layers=2, hidden_size=hidden_size)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(epochs):
    for i, (name, label) in enumerate(zip(X_train,y_train)):
        output = model(name)
        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % print_interval == 0:
            print(
                f"Epoch [{epoch + 1}/{epochs}], "
                f"Step [{i + 1}/{len(X_train)}], "
                f"Loss: {loss.item():.4f}"
            )

Epoch [1/3], Step [1000/16056], Loss: 2.4057
Epoch [1/3], Step [2000/16056], Loss: 0.2275
Epoch [1/3], Step [3000/16056], Loss: 2.3151
Epoch [1/3], Step [4000/16056], Loss: 0.0010
